In [1]:
%matplotlib inline
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import analysis as analysis
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
import linReg as linReg

In [2]:
#df = analysis.read_pickle('df_sayama_change_all')
df = analysis.read_pickle('df_base')
con = df['giantComponent'] < 1.0
df.loc[con].shape

(569, 18)

## setup dataframe
- remove tuples from cols
- reduce df to analysis rows


In [3]:
df_new = analysis.data_cleanup(df)
#check for correctness
mask = df_new['SPL'].apply(lambda x: isinstance(x, tuple))
mask2 = df_new['diam'].apply(lambda x: isinstance(x, tuple))
print(sum(mask), sum(mask2))
df_new['comps'].max()

0 0


4

In [4]:
#now delete runs wiht multiple components
df = df_new.loc[df['comps'] == 1]
print(df.shape, df['comps'].max())

(21031, 19) 1


In [5]:
df.columns

Index(['degrees', 'clusterCoeff', 'reciprocity', 'center_1', 'center_2',
       'mean_centers', 'overall_mean_culture', 'giantComponent', 'diam', 'SPL',
       'CD', 'c1_init', 'c2_init', 'c_avg_init', 'std_d', 'std_rs', 'std_rw',
       'tags', 'comps'],
      dtype='object')

In [6]:
linReg.fit_model(df, "sayama-base")

## CD regression

In [7]:
#fit model
res = smf.ols(formula='CD ~ (std_d + std_rs + std_rw) ** 2', data=df).fit()
print(type(res.summary()))
print(anova_lm(res))

<class 'statsmodels.iolib.summary.Summary'>
                    df       sum_sq     mean_sq            F         PR(>F)
std_d              1.0   752.778129  752.778129  3282.411539   0.000000e+00
std_rs             1.0   976.525329  976.525329  4258.038172   0.000000e+00
std_rw             1.0   333.500273  333.500273  1454.193611  1.006164e-307
std_d:std_rs       1.0   495.219666  495.219666  2159.354377   0.000000e+00
std_d:std_rw       1.0   141.646281  141.646281   617.634025  2.122770e-134
std_rs:std_rw      1.0   152.785121  152.785121   666.203793  1.193131e-144
Residual       21024.0  4821.579255    0.229337          NaN            NaN


## SPL regression

In [8]:
#fit model
res = smf.ols(formula='SPL ~ (std_d + std_rs + std_rw) ** 2', data=df).fit()
print(res.summary())
print(anova_lm(res))

                            OLS Regression Results                            
Dep. Variable:                    SPL   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     2640.
Date:                Thu, 16 Jul 2020   Prob (F-statistic):               0.00
Time:                        13:04:51   Log-Likelihood:                 12926.
No. Observations:               21031   AIC:                        -2.584e+04
Df Residuals:                   21024   BIC:                        -2.578e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.4783      0.004    600.555